In [93]:
import networkx as nx
import pandas as pd
import numpy as np
from sklearn.feature_selection import mutual_info_classif
from pathlib import Path
from frlearn.classifiers import FRNN
from sklearn.model_selection import train_test_split
from frlearn.base import probabilities_from_scores
from sklearn.metrics import roc_auc_score
from frlearn.neighbours.instance_preprocessors import FRPS
import time

from Ant import Colony


In [3]:
class 

In [90]:
Colony.initialize_colony(5, 3, with_core=True, core_accuracy=0.75) 

9
21
33
[9, 21, 33]
ant_route: [19, 22, 6]
ant_instance.fg: [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]
ant_route: [13, 32, 18]
ant_instance.fg: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36]
ant_route: [12, 7, 15]
ant_instance.fg: [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]
ant_route: [1, 0, 29]
ant_instance.fg: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36]
ant_route: [28, 9, 15]
ant_instance.fg: [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 36]
initiall final: [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 

In [91]:
Colony.generate_next_ants(number_of_ants_next_generation=4,
                          q=0.8,
                          phero_rate_decay=0.5,
                          THRESHOLD=[0.95, 'landa', 0.80],
                          alpha=0.8,
                          beta=0.01,
                          with_core=True,
                          change_pheromone=True)
# Colony.generate_next_ants(4, 1, 0.5, [0.97, 'landa', 0.90], 0.8, 0.2)
# Colony.generate_next_ants(4, 1, 0.5, [0.97, 'landa', 0.95], 0.8, 0.2)

cls.fg_core in ant class: [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36]
cls.fg_core in ant class: [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 34, 35, 36]
cls.fg_core in ant class: [0, 1, 2, 3, 4, 5, 6, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 34, 35, 36]
cls.fg_core in ant class: [0, 1, 2, 3, 4, 5, 6, 8, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 34, 35, 36]
cls.fg_core in ant class: [0, 1, 2, 3, 4, 5, 6, 8, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 34, 35]
cls.fg_core in ant class: [0, 1, 2, 3, 4, 5, 6, 8, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 30, 31, 34, 35]
cls.fg_core in ant class: [0, 1, 2, 3, 4, 5, 6, 8, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 30, 34, 35]
c

KeyboardInterrupt: 

In [45]:
Colonies.overall_ant_route

{'Colony Number:2 ant-num:0': [9], 'Colony Number:2 ant-num:1': [21]}

In [123]:
print(f'initial : {Colonies.overall_ant_route_init}')
print(f'next GEN : {Colonies.overall_ant_route}')

initial : {0: [31, 26, 0], 1: [15, 24, 30], 2: [23, 2, 0], 3: [7, 30, 1], 4: [2, 14, 1]}
next GEN : {'Colony Number:2 ant-num:0': [21, 9, 33, 32, 7, 15, 36, 29, 31, 16, 18, 6, 23, 14, 35, 1, 27, 22, 5, 0], 'Colony Number:2 ant-num:1': [21, 9, 33, 32, 7, 15, 36, 29, 31, 16, 18, 6, 23, 14, 35, 1, 27, 22, 5, 0]}


In [125]:
np.set_printoptions(threshold=np.inf)
print(f'pheromone : {Colonies.pheromone}') 

pheromone : [[0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5 ]
 [0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.54
  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5 ]
 [0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5 ]
 [0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5 ]
 [0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5 ]
 [0.54 0.5  0.5  0.5  0.5  0.5  0.5  

# transfer to GAN

In [5]:
D = Lf*(1 + landa)
Lf = max(ant_route accuracy)
G = Li
if Li < D:
    change_phromone()

In [44]:
len(ns)

3196

In [49]:

X = ns.iloc[:, [36, 1, 2]].values
y = Y.values.squeeze()
clf = FRPS()
## FRPS() choose instances that are not ROUGH!
## instances with quality measure more than maximum tau
selected_dataset = clf(X, y)
print(len(selected_dataset[0])/len(ns))
# len(selected_dataset[1])

0.6874217772215269


In [46]:
len(selected_dataset[0])

3196

In [21]:
X = ns.iloc[:, [32]].values
y = Y.values.squeeze()
clf = FRPS()
## FRPS() choose instances that are not ROUGH!
## instances with quality measure more than maximum tau
selected_dataset = clf(X, y)
len(selected_dataset)/len(X)
len(selected_dataset[0])
# round(625 / len(X),3)

1824

In [14]:
selected_dataset[]
len(selected_dataset[1])

3196

In [88]:
Colony.log

{0: [32, 21],
 1: [0, 21],
 2: [30, 21],
 3: [36, 21],
 4: [11, 12, 6],
 'landa': 0.840738423028786}